In [6]:
import pandas as pd

super_dataset = pd.read_csv('../../../data/stage_I/superDataset.csv')
world_dataset = pd.read_csv('../../../data/stage_II/owid-covid-data.csv')

countries = ['Indonesia', 'Pakistan', 'Nigeria', 'Brazil', 'Bangladesh']

In [7]:
filtered_dataset = world_dataset[world_dataset.location.isin(countries)]
filtered_dataset

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
5689,BGD,Asia,Bangladesh,2020-03-03,NaN,NaN,NaN,NaN,NaN,NaN,...,3523.984,14.8,298.003,8.38,1.0,44.7,34.808,0.8,72.59,0.632
5690,BGD,Asia,Bangladesh,2020-03-04,NaN,NaN,NaN,NaN,NaN,NaN,...,3523.984,14.8,298.003,8.38,1.0,44.7,34.808,0.8,72.59,0.632
5691,BGD,Asia,Bangladesh,2020-03-05,NaN,NaN,NaN,NaN,NaN,NaN,...,3523.984,14.8,298.003,8.38,1.0,44.7,34.808,0.8,72.59,0.632
5692,BGD,Asia,Bangladesh,2020-03-06,NaN,NaN,NaN,NaN,NaN,NaN,...,3523.984,14.8,298.003,8.38,1.0,44.7,34.808,0.8,72.59,0.632
5693,BGD,Asia,Bangladesh,2020-03-07,NaN,NaN,NaN,NaN,NaN,NaN,...,3523.984,14.8,298.003,8.38,1.0,44.7,34.808,0.8,72.59,0.632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50163,PAK,Asia,Pakistan,2021-02-28,581365.0,1392.0,1290.143,12896.0,36.0,39.857,...,5034.708,4.0,423.031,8.35,2.8,36.7,59.607,0.6,67.27,0.557
50164,PAK,Asia,Pakistan,2021-03-01,582528.0,1163.0,1306.286,12938.0,42.0,40.000,...,5034.708,4.0,423.031,8.35,2.8,36.7,59.607,0.6,67.27,0.557
50165,PAK,Asia,Pakistan,2021-03-02,583916.0,1388.0,1333.714,13013.0,75.0,43.571,...,5034.708,4.0,423.031,8.35,2.8,36.7,59.607,0.6,67.27,0.557
50166,PAK,Asia,Pakistan,2021-03-03,585435.0,1519.0,1356.286,13076.0,63.0,43.429,...,5034.708,4.0,423.031,8.35,2.8,36.7,59.607,0.6,67.27,0.557


In [8]:
grouped_dataset = world_dataset.groupby('location')

for location, country in grouped_dataset:
    country.